In [1]:
import pandas as pd
import re

In [2]:
dataset = pd.read_csv('/kaggle/input/sms-data/SMS-Data.csv')
dataset = dataset['text']

In [3]:
dataset.head()

0    Rs.95.15 on Zomato charged via Simpl.\r\n--\r\...
1    Hi! Update your email id through WhatsApp: htt...
2    Lucknow ya Kolkata ? - watch it LIVE with Vi c...
3    Mohd,\nCheck the incredible Acko insurance pol...
4    Hi! You can now get your Vi prepaid invoice em...
Name: text, dtype: object

In [4]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 67.1 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
dataset_arr = []
for sms in dataset:
    sms = re.sub(r'\s', ' ', str(sms))
    sms = re.sub(r'-', ' ', sms)
    sms = re.sub(r'\s+', ' ', sms)
    dataset_arr.append(sms)

In [6]:
dataset_arr[0]

'Rs.95.15 on Zomato charged via Simpl. Food, groceries, commute, or medicines. Buy Now, Pay Later via Simpl. Know More: https://click.getsimpl.com/vyhm/5b611f85 Simpl Pay'

In [7]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
for i in range(20):
    labels = nlp(dataset_arr[i])
    displacy.render(labels, style="ent", jupyter=True)

/opt/conda/lib/python3.7/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [8]:
len(dataset_arr)

100243

In [9]:
import json
import os
    
with open('/kaggle/input/sms-ner-dataset/annotations.json', 'r') as file:
    data = json.load(file)
    
# with open('/kaggle/input/sms-ner-dataset-new/annotations_new.json', 'r') as file:
#     data = json.load(file)

In [10]:
data = data['annotations']
data = [tuple(i) for i in data]

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 41)

In [11]:
train_data[0]

('b"Last few hours to apply for BYJU\'S Aptitude Test Win up to 100% Scholarship Get an all India percentile & more Apply Now http://bit.ly/BNAT Scholarship 5"',
 {'entities': [[30, 36, 'TITLE'], [37, 50, 'PURPOSE'], [61, 77, 'PURPOSE']]})

In [12]:
for i in train_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, 'PERSON')
    else:
        for j in range(len(i[1]['entities'])):
            i[1]['entities'][j] = tuple(i[1]['entities'][j])

In [13]:
for i in test_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, 'PERSON')
    else:
        for j in range(len(i[1]['entities'])):
            i[1]['entities'][j] = tuple(i[1]['entities'][j])

In [14]:
print(len(train_data), len(test_data))

88 22


In [15]:
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

def make_doc_for_data(data):
    nlp = spacy.load("en_core_web_sm")

    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        if(len(text) > 512):
            continue
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
        
    return db

make_doc_for_data(train_data).to_disk("train.spacy") # save the docbin object
make_doc_for_data(test_data).to_disk("test.spacy") # save the docbin object

100%|██████████| 88/88 [00:00<00:00, 1837.22it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 22/22 [00:00<00:00, 1198.79it/s]

Skipping entity


In [16]:
!pip install spacy_transformers
!python -m spacy init fill-config /kaggle/input/sms-ner-dataset/base_config.cfg config.cfg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.7/191.7 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.6 MB/s eta 0:00:0000:01
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [17]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-09 05:28:31,575] [INFO] Set up nlp object from config
[2023-05-09 05:28:31,588] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-09 05:28:31,593] [INFO] Created vocabulary
[2023-05-09 05:28:31,595] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be

In [18]:
import spacy_transformers
nlp1 = spacy.load('/kaggle/working/output/model-best') #load the best model
doc = nlp1(dataset_arr[1000]) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [19]:
for i in range(1505, 1515):
    doc1 = nlp1(dataset_arr[i]) # input sample text
    spacy.displacy.render(doc1, style="ent", jupyter=True) # display in Jupyter

/opt/conda/lib/python3.7/site-packages/spacy/displacy/__init__.py:211: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [20]:
# Download model-best for future use
import shutil
shutil.make_archive('output', 'zip', '/kaggle/working/')

'/kaggle/working/output.zip'